In [4]:
import os
import numpy as np
import pandas as pd
import pickle
import time
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim

from torch_geometric.data import Dataset
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn import global_mean_pool, global_max_pool, global_add_pool

from torch_geometric.utils import scatter

import time
import wandb
from tqdm import tqdm
from collections import Counter

import sys
sys.path.insert(1, 'data/')
from pyg_dataset import NetlistDataset

sys.path.append("models/layers/")
from models.model_att import GNN_node
from sklearn.metrics import accuracy_score, precision_score, recall_score




# Function to compute accuracy, precision, and recall
def compute_metrics(true_labels, predicted_labels):
    # Accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    
    # Precision
    precision = precision_score(true_labels, predicted_labels, average='binary')
    
    # Recall
    recall = recall_score(true_labels, predicted_labels, average='binary')
    
    return accuracy, precision, recall

### hyperparameter ###
test = False # if only test but not train
restart = False # if restart training
reload_dataset = True # if reload already processed h_dataset

if test:
    restart = True

prediction = 'congestion' # one of ['congestion', 'demand']
model_type = "dehnn" #this can be one of ["dehnn", "dehnn_att", "digcn", "digat"] "dehnn_att" might need large memory usage
num_layer = 2 #large number will cause OOM
num_dim = 16 #large number will cause OOM
vn = True #use virtual node or not
trans = False #use transformer or not
aggr = "add" #use aggregation as one of ["add", "max"]
device = "cpu" #use cuda or cpu
learning_rate = 0.001
num_epochs = 200

if not reload_dataset:
    dataset = NetlistDataset(data_dir="../data/superblue", load_pe = True, pl = True, processed = reload_dataset, load_indices=None)
    h_dataset = []
    for data in tqdm(dataset):
        num_instances = data.node_features.shape[0]
        data.num_instances = num_instances
        data.edge_index_sink_to_net[1] = data.edge_index_sink_to_net[1] - num_instances
        data.edge_index_source_to_net[1] = data.edge_index_source_to_net[1] - num_instances
        
        out_degrees = data.net_features[:, 0]
        mask = (out_degrees < 3000)
        mask_edges = mask[data.edge_index_source_to_net[1]] 
        filtered_edge_index_source_to_net = data.edge_index_source_to_net[:, mask_edges]
        data.edge_index_source_to_net = filtered_edge_index_source_to_net

        mask_edges = mask[data.edge_index_sink_to_net[1]] 
        filtered_edge_index_sink_to_net = data.edge_index_sink_to_net[:, mask_edges]
        data.edge_index_sink_to_net = filtered_edge_index_sink_to_net

        h_data = HeteroData()
        h_data['node'].x = data.node_features
        h_data['net'].x = data.net_features.float()

        
        edge_index = torch.concat([data.edge_index_sink_to_net, data.edge_index_source_to_net], dim=1)
        h_data['node', 'to', 'net'].edge_index, h_data['node', 'to', 'net'].edge_weight = gcn_norm(edge_index, add_self_loops=False)
        h_data['node', 'to', 'net'].edge_type = torch.concat([torch.zeros(data.edge_index_sink_to_net.shape[1]), torch.ones(data.edge_index_source_to_net.shape[1])]).bool()
        h_data['net', 'to', 'node'].edge_index, h_data['net', 'to', 'node'].edge_weight = gcn_norm(edge_index.flip(0), add_self_loops=False)
        
        h_data['design_name'] = data['design_name']
        h_data.num_instances = data.node_features.shape[0]
        variant_data_lst = []

        node_congestion = data.node_congestion.long()
        net_congestion = data.net_congestion.long()
        node_demand = data.node_demand.float()
        net_demand = data.net_demand.float()
        node_demand = (node_demand - torch.mean(node_demand)) / torch.std(node_demand)
        net_demand = (net_demand - torch.mean(net_demand))/ torch.std(net_demand)
        
        batch = data.batch
        num_vn = len(np.unique(batch))
        vn_node = torch.concat([global_mean_pool(h_data['node'].x, batch), 
                global_max_pool(h_data['node'].x, batch)], dim=1)

        variant_data_lst.append((node_demand, net_demand, node_congestion, net_congestion, batch, num_vn, vn_node)) 
        h_data['variant_data_lst'] = variant_data_lst
        h_dataset.append(h_data)
        
    torch.save(h_dataset, "h_dataset.pt")
    
else:
    dataset = torch.load("h_dataset.pt")
    h_dataset = []
    for data in dataset:
        h_dataset.append(data)
    
sys.path.append("models/layers/")

h_data = h_dataset[0]
if restart:
    model = torch.load(f"{model_type}_{num_layer}_{num_dim}_{vn}_{trans}_model.pt")
elif prediction == 'congestion':
    model = GNN_node(num_layer, num_dim, 2, 2, node_dim = h_data['node'].x.shape[1], net_dim = h_data['net'].x.shape[1], gnn_type=model_type, vn=vn, trans=trans, aggr=aggr, JK="Normal").to(device)
else:
    model = GNN_node(num_layer, num_dim, 1, 1, node_dim = h_data['node'].x.shape[1], net_dim = h_data['net'].x.shape[1], gnn_type=model_type, vn=vn, trans=trans, aggr=aggr, JK="Normal").to(device)

if prediction == 'congestion':
    criterion_node = nn.CrossEntropyLoss()
    criterion_net = nn.CrossEntropyLoss()
else:
    criterion_node = nn.MSELoss()
    criterion_net = nn.MSELoss()

optimizer = optim.AdamW(model.parameters(), lr=learning_rate,  weight_decay=0.01)
load_data_indices = [idx for idx in range(len(h_dataset))]
all_train_indices, all_valid_indices, all_test_indices = load_data_indices[:10], load_data_indices[10:11], load_data_indices[11:12]
best_total_val = None

now = datetime.now()
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
filepath = f"../results/baselines/{timestamp}__{num_epochs}_{num_layer}_{num_dim}_{vn}_{prediction}_baseline.csv"
with open(filepath, 'a') as f:
    if prediction == 'congestion':
        f.write('Epoch,TruePositive,FalsePositive,TrueNegative,FalseNegative,Precision,Recall,Fscore,NodeTrain,NetTrain,NodeValid,NetValid,Time\n')
    else:
        f.write('Epoch,NodeTrain,NetTrain,NodeValid,NetValid,Time\n')

if not test:
    start_time = time.time()
    for epoch in range(num_epochs):
        np.random.shuffle(all_train_indices)
        loss_node_all = 0
        loss_net_all = 0
        val_loss_node_all = 0
        val_loss_net_all = 0
        
        all_train_idx = 0
        for data_idx in tqdm(all_train_indices):
            data = h_dataset[data_idx]
            for inner_data_idx in range(len(data.variant_data_lst)):
                target_node_demand, target_net_demand, target_node_congestion, target_net_congestion, batch, num_vn, vn_node = data.variant_data_lst[inner_data_idx]
                optimizer.zero_grad()
                data.batch = batch
                data.num_vn = num_vn
                data.vn = vn_node
                node_representation, net_representation = model(data, device)
                node_representation = torch.squeeze(node_representation).float()
                net_representation = torch.squeeze(net_representation).float()

                # print(node_representation)
                # print(target_node_demand)

                if prediction == 'congestion':
                    loss_node = criterion_node(node_representation, target_node_congestion.to(device))
                    loss_net = criterion_net(net_representation, target_net_congestion.to(device))
                else:
                    loss_node = criterion_node(node_representation, target_node_demand.to(device))
                    loss_net = criterion_net(net_representation, target_net_demand.to(device))
                
                loss = loss_node + loss_net
                # loss = loss_net
                loss.backward()
                optimizer.step()   
    
                loss_node_all += loss_node.item()
                loss_net_all += loss_net.item()
                all_train_idx += 1
        print('Training Loss Node: ', loss_node_all/all_train_idx, ', Net: ', loss_net_all/all_train_idx)
    
        all_valid_idx = 0
        for data_idx in tqdm(all_valid_indices):
            data = h_dataset[data_idx]
            for inner_data_idx in range(len(data.variant_data_lst)):
                target_node_demand, target_net_demand, target_node_congestion, target_net_congestion, batch, num_vn, vn_node = data.variant_data_lst[inner_data_idx]
                data.batch = batch
                data.num_vn = num_vn
                data.vn = vn_node
                node_representation, net_representation = model(data, device)
                node_representation = torch.squeeze(node_representation)
                net_representation = torch.squeeze(net_representation)
                
                # val_loss_node = criterion_node(node_representation, target_node.to(device))
                # val_loss_net = criterion_net(net_representation, target_net.to(device))
                if prediction == 'congestion':
                    val_loss_node = criterion_node(node_representation, target_node_congestion.to(device))
                    val_loss_net = criterion_net(net_representation, target_net_congestion.to(device))
                else:
                    val_loss_node = criterion_node(node_representation, target_node_demand.to(device))
                    val_loss_net = criterion_net(net_representation, target_net_demand.to(device))

                val_loss_node_all +=  val_loss_node.item()
                val_loss_net_all += val_loss_net.item()
                all_valid_idx += 1

                if prediction == 'congestion':
                    outs = node_representation.detach().numpy()
                    pred_vals = np.array([0 if i > j else 1 for i, j in outs])
                    real_vals = target_node_congestion.numpy()
                    tp, fp, tn, fn = 0, 0, 0, 0
                    for i, j in zip(pred_vals, real_vals):
                        if i == 1 and j == 1:
                            tp += 1
                        elif i == 1 and j == 0:
                            fp += 1
                        elif i == 0 and j == 1:
                            fn += 1
                        else:
                            tn += 1
                    p = 0
                    r = 0
                    fsc = 0
                    if tp + fp > 0:
                        p = tp / (tp + fp)
                    if tp + fn > 0:
                        r = tp / (tp + fn)
                    if p + r > 0:
                        fsc = (2 * p * r) / (p + r)

        print('Validation Loss Node: ', val_loss_node_all/all_valid_idx, ', Net: ',  val_loss_net_all/all_valid_idx)
        if prediction == 'congestion':
            print(f'Precision: {p}, Recall: {r}, F-score: {fsc}')
            print(tp, fp)
            print(fn, tn)
        print(f'Epoch {epoch+1}, {round(time.time() - start_time, 2)}s\n')
    
        if (best_total_val is None) or ((loss_node_all/all_train_idx) < best_total_val):
            best_total_val = loss_node_all/all_train_idx
            torch.save(model, f"{model_type}_{num_layer}_{num_dim}_{vn}_{trans}_{prediction}.pt")

        with open(filepath, 'a') as f:
            if prediction == 'congestion':
                f.write(f'{epoch+1},{tp},{fp},{tn},{fn},{p},{r},{fsc},')
                f.write(f'{loss_node_all/all_train_idx},{loss_net_all/all_train_idx},{val_loss_node_all/all_valid_idx},{val_loss_net_all/all_valid_idx},{round(time.time()-start_time, 2)}\n')
            else:
                f.write(f'{epoch+1},{loss_node_all/all_train_idx},{loss_net_all/all_train_idx},{val_loss_node_all/all_valid_idx},{val_loss_net_all/all_valid_idx},{round(time.time()-start_time, 2)}\n')
        
else:
    all_test_idx = 0
    test_loss_node_all = 0
    test_loss_net_all = 0
    for data_idx in tqdm(all_test_indices):
        data = h_dataset[data_idx]
        for inner_data_idx in range(len(data.variant_data_lst)):
            target_node, target_net_hpwl, target_net_demand, batch, num_vn, vn_node = data.variant_data_lst[inner_data_idx]
            data.batch = batch
            data.num_vn = num_vn
            data.vn = vn_node
            node_representation, net_representation = model(data, device)
            node_representation = torch.squeeze(node_representation)
            net_representation = torch.squeeze(net_representation)
            
            test_loss_node = criterion_node(node_representation, target_node.to(device))
            test_loss_net = criterion_net(net_representation, target_net_demand.to(device))
            test_loss_node_all +=  test_loss_node.item()
            test_loss_net_all += test_loss_net.item()
            all_test_idx += 1
    print("avg test node demand mse: ", test_loss_node_all/all_test_idx)
    print("avg test net demand mse: ", test_loss_net_all/all_test_idx)

C:\Users\rebal\AppData\Local\Temp\ipykernel_32248\3473235973.py:120: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load("h_dataset.pt")
  0%|          | 0/10

ValueError: not enough values to unpack (expected 7, got 6)

In [5]:
h_dataset

[HeteroData(
   design_name='superblue_1',
   num_instances=797938,
   variant_data_lst=[1],
   node={ x=[797938, 45] },
   net={ x=[821523, 11] },
   (node, to, net)={
     edge_index=[2, 2805327],
     edge_weight=[2805327],
     edge_type=[2805327],
   },
   (net, to, node)={
     edge_index=[2, 2805327],
     edge_weight=[2805327],
   }
 ),
 HeteroData(
   design_name='superblue_11',
   num_instances=923355,
   variant_data_lst=[1],
   node={ x=[923355, 45] },
   net={ x=[954144, 11] },
   (node, to, net)={
     edge_index=[2, 3049383],
     edge_weight=[3049383],
     edge_type=[3049383],
   },
   (net, to, node)={
     edge_index=[2, 3049383],
     edge_weight=[3049383],
   }
 ),
 HeteroData(
   design_name='superblue_14',
   num_instances=604921,
   variant_data_lst=[1],
   node={ x=[604921, 45] },
   net={ x=[627036, 11] },
   (node, to, net)={
     edge_index=[2, 2013969],
     edge_weight=[2013969],
     edge_type=[2013969],
   },
   (net, to, node)={
     edge_index=[2, 2013

In [6]:
len(h_dataset)

12

In [3]:
node_representation

tensor([[-0.3972,  0.2553],
        [-0.3541,  0.2041],
        [-0.3527,  0.2020],
        ...,
        [-0.3042,  0.1830],
        [-0.3047,  0.1807],
        [-0.9496,  0.5788]], grad_fn=<AddmmBackward0>)

In [4]:
target_node.numpy().mean()

0.5329729465833162

In [5]:
outs = node_representation.detach().numpy()
outs

array([[-0.3972408 ,  0.25528955],
       [-0.35414067,  0.20412536],
       [-0.35266522,  0.20200948],
       ...,
       [-0.3041961 ,  0.18303321],
       [-0.30473903,  0.18074541],
       [-0.9495648 ,  0.5788216 ]], dtype=float32)

In [6]:
pred_vals = np.array([0 if i > j else 1 for i, j in outs])
pred_vals

array([1, 1, 1, ..., 1, 1, 1])

In [7]:
real_vals = target_node.numpy()
real_vals

array([0, 1, 1, ..., 0, 1, 1], dtype=int64)

In [8]:
tp, fp, tn, fn = 0, 0, 0, 0
for i, j in zip(pred_vals, real_vals):
    if i == 1 and j == 1:
        tp += 1
    elif i == 1 and j == 0:
        fp += 1
    elif i == 0 and j == 1:
        fn += 1
    else:
        tn += 1

In [9]:
print(tp, fp, '\n', fn, tn)

462571 399711 
 17773 21199


In [10]:
# Precision
p = tp / (tp + fp)
p

0.5364497925272707

In [11]:
# Recall
r = tp / (tp + fn)
r

0.9629994337391536

In [12]:
(2 * p * r) / (p + r)

0.6890541371908484